Install packages and dependencies

In [2]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install sentence-transformers
!pip install -U langchain-community
!pip install ragatouille
!pip install tqdm openpyxl pandas
!pip install ipywidgets
!pip install datasets
!pip install frontend
!pip install huggingface_hub
!pip install json
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.3/311.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.

In [3]:
from tqdm.auto import tqdm

import pandas as pd

from typing import Optional, List, Tuple

import json

import datasets

import os

pd.set_option("display.max_colwidth", None)

In [4]:
from huggingface_hub.hf_api import HfFolder
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HfFolder.save_token(user_secrets.get_secret("HF_TOKEN"))

In [5]:
# import json

# # Path to your JSONL file
# jsonl_path = '/kaggle/input/scrapped-dataset/results.jsonl'

# # Load JSONL data
# def load_jsonl_data(filepath):
#     data = []
#     with open(filepath, 'r') as f:
#         for line in f:
#             data.append(json.loads(line))
#     return data

# # Load the data
# data = load_jsonl_data(jsonl_path)


In [6]:
from langchain.prompts import PromptTemplate

from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


import os

import bs4


from getpass import getpass

from uuid import uuid4

from langchain.docstore.document import Document as LangchainDocument

from langchain.text_splitter import RecursiveCharacterTextSplitter



text_splitter = RecursiveCharacterTextSplitter(

    chunk_size=900,

    chunk_overlap=150,

    add_start_index=True,

    separators=["\n\n", "\n", ".", " ", ""],

)


In [7]:
# Path to JSONL file
jsonl_path = '/kaggle/input/scrapped-dataset/results.jsonl'

# Load JSONL data
def load_jsonl_data(filepath):
    data = []
    with open(filepath, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# Read JSONL data and create LangchainDocument objects for each entry
data = load_jsonl_data(jsonl_path)

In [8]:
# List to store extracted documents
docs = []
for entry in data:
    # Use the 'content' field from each JSON entry as document text
    text = entry.get('content', '')
    if text:  # Ensure there's text to process
        docs.append(LangchainDocument(page_content=text))

# Process documents with the text splitter
docs_processed = []
for doc in docs:
    docs_processed += text_splitter.split_documents([doc])

In [9]:
docs_processed

[Document(metadata={'start_index': 0}, page_content='Visa'),
 Document(metadata={'start_index': 0}, page_content='Visa\n\nD-2 (Overseas Study) VISA\n\nInternational Students must apply if he/she does not have a proper visa to study in Korea.<Required Documents>Application form\n\na copy of the Passport\n\nPassport-sized color photo (1 copy)\n\nCertificate of Admission\n\nFinancial Statement(Bank statement or Scholarship certificate)\n\nCertificate of Graduation with Appostile or Korean Consular-confirmation stamp\nOthersForeigner Registration-\xa0 New international students who hold\xa0D-2 visa are obligated to register (or report) their status in Korea.\n-\xa0 Registration period: Within 90 days from the entry date<Requried Documents>\n\n\nApplication form (download from\xa0www.hikorea.go.kr)\nA copy of the passport\nPassport-sized color photo (1 copy)\nCertificate of University registration\nReceipt of tuition payment\nProof of residence Document\nFee (30,000 Won)'),
 Document(metada

In [10]:
os.environ["OPENAI_API_KEY"] = user_secrets.get_secret("Open_AI")

In [11]:
from openai import OpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain


client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  
)

def call_llm(prompt: str):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "assistant",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
    )
    return response.choices[0].message.content

# Example call
print(call_llm("This is a test context"))

Sure! How can I assist you today?


In [12]:
QA_generation_prompt = """

Your task is to write a factoid question and an answer given a context.

Your factoid question should be answerable with a specific, concise piece of factual information from the context.

Your factoid question should be formulated in the same style as questions users could ask in a search engine.

Your factoid question should be related to Sungkyunkwan University (성균관대학교).

This means that your factoid question MUST NOT mention something like "according to the passage" or "context".



Provide your answer as follows:



Output:::

Factoid question: (your factoid question)

Answer: (your answer to the factoid question)



Now here is the context.



Context: {context}\n

Output:::"""

In [13]:
docs_processed.__len__()

331

In [14]:
import random

from tqdm import tqdm

N_GENERATIONS = 331 # We intentionally generate only 10 QA couples here for cost and time considerations



print(f"Generating {N_GENERATIONS} QA couples...")



outputs = []

for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):

    # Generate QA couple

    output_QA_couple = call_llm(prompt = QA_generation_prompt.format(context=sampled_context.page_content))

    # print("Generated output:", output_QA_couple)

    try:

        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]

        answer = output_QA_couple.split("Answer: ")[-1]

        assert len(answer) < 300, "Answer is too long"

        outputs.append(

            {

                "content": sampled_context.page_content,

                "question": question,

                "answer": answer,

                # "source_doc": sampled_context.metadata["source"],

            }

        )

    except:

        continue

Generating 331 QA couples...


100%|██████████| 331/331 [07:25<00:00,  1.35s/it]


In [15]:
display(pd.DataFrame(outputs).head(3))

,content,question,answer
0,Course\nCourses\nExams\n\nThe dates and details about online courses will be posted on the GLS CYAN notice and icampus notice boards every semester before the semester begins.,When are the dates and details about online courses posted at Sungkyunkwan University?\n\n,The dates and details about online courses are posted every semester before the semester begins.
1,"},\n {\n ""Region"": ""Seoul"",\n ""Location"": ""Seoul South Immigration Office\n\n\nJurisdiction : Seoul (Seodaemun-gu, Mapo-gu, Gangseo-gu, Yangcheon-gu, Yeongdeungpo-gu, Guro-gu, Geumcheon-gu) Gyeonggi-do(Gwangmyeong-si)\n\nLocation : 319-2, Sinjeong 6 dong, Yangcheon-gu, Seoul\n\nSubway : Line5 Omokgyo Station Exit 7 OR Mok-dong Station Exit 5,6""\n },\n {\n ""Region"": ""Suwon"",\n ""Location"": ""Suwon Immigration Office\n\n\nJurisdiction : Gyeonggi-do(Gunpo-si, Uiwang-si, Suwon-si, Yongin-si, Icheon-si, Hwaseong-si, Gwangju-si,(Yangpyeong-gun, Yeoju-gun)\n\nLocation : 1012-6, Yeong-tong dong, Yeong-tong gu, Suwon\n\nSubway : Exit 1, Yeongtong Station, Line 1""\n },\n {\n ""Region"": ""Suwon"",",Where is Sungkyunkwan University located?\n\n,"Seoul, South Korea"
2,"""Eligible student"": ""Enrolled 9 semesters or more\nDept. of Architecture(11 or more)"",\n ""Registered Credits"": ""10 or more"",\n ""Invoiced Amount"": ""full tuition fee""\n },\n {\n ""Degree"": ""General Graduate School"",\n ""Eligible student"": ""Enrolled 5 semesters or more\nCombined Master/Ph.D.(9 or more)"",\n ""Registered Credits"": ""3 or less"",\n ""Invoiced Amount"": ""1/2 of tuition fee""\n },\n {\n ""Degree"": ""General Graduate School"",\n ""Eligible student"": ""Enrolled 5 semesters or more\nCombined Master/Ph.D.(9 or more)"",\n ""Registered Credits"": ""4 or more"",\n ""Invoiced Amount"": ""Full tuition fee""\n },\n {\n ""Degree"": ""Professional Graduate School"",\n ""Eligible student"": ""Enrolled 5 semesters or more\nCombined Master/Ph.D.(9 or more)\nCombined Master/Ph.D. at the College of Medicine(15 or more)"",",What is the invoiced amount for students in the General Graduate School who are enrolled for 5 semesters or more with 4 or more registered credits?\n\n,Full tuition fee


In [16]:
question_groundedness_critique_prompt = """

You will be given a context and a question related to Sungkyunkwan University (성균관대학교).

Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.

Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.



Provide your answer as follows:



Answer:::

Evaluation: (your rationale for the rating, as a text)

Total rating: (your rating, as a number between 1 and 5)



You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.



Now here are the question and context.



Question: {question}\n

Context: {context}\n

Answer::: """



question_relevance_critique_prompt = """

You will be given a question related to Sungkyunkwan University (성균관대학교).

Your task is to provide a 'total rating' representing how useful this question can be to students and more specifically exchange students of Sungkyunkwan University (성균관대학교) .

Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.



Provide your answer as follows:



Answer:::

Evaluation: (your rationale for the rating, as a text)

Total rating: (your rating, as a number between 1 and 5)



You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.



Now here is the question.



Question: {question}\n

Answer::: """

In [17]:
print("Generating critique for each QA couple...")

for output in tqdm(outputs):

    evaluations = {

        "groundedness": call_llm(

            prompt = question_groundedness_critique_prompt.format(context=output["content"], question=output["question"]),

        ),

        "relevance": call_llm(

            prompt = question_relevance_critique_prompt.format(question=output["question"]),

        ),

    }

    try:

        for criterion, evaluation in evaluations.items():

            score, eval = (

                int(evaluation.split("Total rating: ")[-1].strip()),

                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],

            )

            output.update(

                {

                    f"{criterion}_score": score,

                    f"{criterion}_eval": eval,

                }

            )

    except Exception as e:

        continue

Generating critique for each QA couple...


100%|██████████| 331/331 [23:56<00:00,  4.34s/it]  


In [18]:
print(outputs)

[{'content': 'Course\nCourses\nExams\n\nThe dates and details about online courses will be posted on the GLS CYAN notice and icampus notice boards every semester before the semester begins.', 'question': 'When are the dates and details about online courses posted at Sungkyunkwan University?\n\n', 'answer': 'The dates and details about online courses are posted every semester before the semester begins.', 'groundedness_score': 4, 'groundedness_eval': 'The context clearly states that the dates and details about online courses will be posted on the GLS CYAN notice and icampus notice boards every semester before the semester begins. However, it does not provide specific dates or further details about the posting schedule. Therefore, the question can be answered with general information but lacks specificity regarding exact dates.\n\n', 'relevance_score': 4, 'relevance_eval': "This question is quite useful for students, especially exchange students, as it addresses when and how they can acc

In [22]:
import pandas as pd



pd.set_option("display.max_colwidth", None)

results_df = pd.read_json('/kaggle/input/scrapped-dataset/results.jsonl', lines=True)

# Assume 'outputs' is a list of dictionaries containing 'question' and 'answer'
generated_questions = pd.DataFrame.from_dict(outputs)

# Merge the generated questions with the titles using 'content' as the key
generated_questions = pd.merge(generated_questions, results_df[['title', 'content']], on='content', how='left')
print("Evaluation dataset before filtering:")

display(

    generated_questions[

        [
            "title",
            
            "question",

            "answer",

            "groundedness_score",

            "relevance_score",

        ]

    ]

)

generated_questions = generated_questions.loc[

    (generated_questions["groundedness_score"] >= 3)

    & (generated_questions["relevance_score"] >= 3)

]

print("============================================")

print("Final evaluation dataset:")

display(

    generated_questions[

        [
            "title",
            "question",

            "answer",

            "groundedness_score",

            "relevance_score",

        ]

    ]

)



eval_dataset = datasets.Dataset.from_pandas(generated_questions, split="train", preserve_index=False)

Evaluation dataset before filtering:


,title,question,answer,groundedness_score,relevance_score
0,Course-6,When are the dates and details about online courses posted at Sungkyunkwan University?\n\n,The dates and details about online courses are posted every semester before the semester begins.,4,4
1,NaN,Where is Sungkyunkwan University located?\n\n,"Seoul, South Korea",1,5
2,NaN,What is the invoiced amount for students in the General Graduate School who are enrolled for 5 semesters or more with 4 or more registered credits?\n\n,Full tuition fee,5,3
3,NaN,When did Sungkyunkwan University implement the Core Major Category for mandatory courses?\n\n,In 2013.,5,3
4,NaN,Where is the Sungkyun Language Institute located at Sungkyunkwan University?\n\n,Dasan Hall of Economics 3F,5,4
...,...,...,...,...,...
328,NaN,What is the mandatory course for international students at Sungkyunkwan University in the 2024-2 semester?\n\n,"K-law & Education for Human Rights and Prevention of Sexual Violence, Domestic Violence.",5,5
329,NaN,What scholarship program does Sungkyunkwan University offer for international graduate students in STEM fields?\n\n,"SKKU offers a scholarship program for newly admitted international graduate students in STEM fields that can cover up to 100% of the tuition fee, depending on a scholarship evaluation.",5,5
330,Leave of absence Reinstatement-11,What is required for a student approved for re-entry at Sungkyunkwan University?\n\n,A student approved for re-entry needs to pay for their tuition and complete the course registration as scheduled.,5,5
331,Course-9,What types of courses are primarily offered during the Summer/Winter Semester at Sungkyunkwan University for undergraduate students? \n\n,"Courses suitable for intensive curriculums, including liberal arts/fundamental courses, co-op courses, field practice courses, and outstanding undergraduate research credit courses.",5,5


Final evaluation dataset:


,title,question,answer,groundedness_score,relevance_score
0,Course-6,When are the dates and details about online courses posted at Sungkyunkwan University?\n\n,The dates and details about online courses are posted every semester before the semester begins.,4,4
2,NaN,What is the invoiced amount for students in the General Graduate School who are enrolled for 5 semesters or more with 4 or more registered credits?\n\n,Full tuition fee,5,3
3,NaN,When did Sungkyunkwan University implement the Core Major Category for mandatory courses?\n\n,In 2013.,5,3
4,NaN,Where is the Sungkyun Language Institute located at Sungkyunkwan University?\n\n,Dasan Hall of Economics 3F,5,4
5,Part-time works-10,What types of jobs are students at Sungkyunkwan University limited to under their visa status?\n\n,Students are limited to part-time jobs that can be performed by students according to social norms and internships related to acquiring credits.,3,5
...,...,...,...,...,...
328,NaN,What is the mandatory course for international students at Sungkyunkwan University in the 2024-2 semester?\n\n,"K-law & Education for Human Rights and Prevention of Sexual Violence, Domestic Violence.",5,5
329,NaN,What scholarship program does Sungkyunkwan University offer for international graduate students in STEM fields?\n\n,"SKKU offers a scholarship program for newly admitted international graduate students in STEM fields that can cover up to 100% of the tuition fee, depending on a scholarship evaluation.",5,5
330,Leave of absence Reinstatement-11,What is required for a student approved for re-entry at Sungkyunkwan University?\n\n,A student approved for re-entry needs to pay for their tuition and complete the course registration as scheduled.,5,5
331,Course-9,What types of courses are primarily offered during the Summer/Winter Semester at Sungkyunkwan University for undergraduate students? \n\n,"Courses suitable for intensive curriculums, including liberal arts/fundamental courses, co-op courses, field practice courses, and outstanding undergraduate research credit courses.",5,5


In [23]:
eval_dataset.to_csv('/kaggle/working/eval_dataset_6.csv',index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

427661

In [24]:
import pandas as pd

# Load the dataset
df_eval = pd.read_csv('/kaggle/working/eval_dataset_6.csv')

# Remove rows where 'title' is null
df_eval = df_eval.dropna(subset=['title'])

# Save the cleaned dataset
df_eval.to_csv('eval_dataset_cleaned.csv', index=False)
